### 第一步：玩家基本信息查询
首先请运行[basic_analyze_process.sql](basic_analyze_process.sql)，进行基础查询,包括：<br>
1.  统计不同社区可见性状态的用户数量
2.  统计国籍分布情况
3.  计算不同国家玩家的平均游戏数量

### 第二步：热门游戏查询
使用pandas对[STEAM数据文件](../../data/processed/all_steam_and_game_data_after_cleaned.csv)进行更为细致的查询

In [478]:
import pandas as pd
import re
import ast # Abstract Syntax Tree，模块允许Python程序处理、解析和分析Python源代码
data = pd.read_csv('../../data/processed/all_steam_and_game_data_after_cleaned.csv')

#### 统计一直以来的热门游戏

In [479]:
# 使用正则表达式提取playtime列以及对应的游戏时长和价格
selected_columns = []

for idx, col in enumerate(data.columns):
    if re.search(r'playtime', col):
        if idx <= 66:
            # 提取总时长前五的所有游戏信息
            selected_columns.extend([data.columns[idx-1], data.columns[idx], data.columns[idx+1],data.columns[idx+2],data.columns[idx+3],data.columns[idx+4], data.columns[idx+5], data.columns[idx+6]]) 
        else:
            pass


# 创建包含这些列的新DataFrame
new_df = data[selected_columns]


In [480]:
games_df = pd.DataFrame()
# 在进行concate时，要保证列名一致，因此此处清空列名
new_df.rename(columns={col:''  for col in new_df.columns}, inplace=True) 

for i in range(0, len(new_df.columns), 8):   

    subset = new_df.iloc[:, i:i+8] # 选择当前组的8列
    
    # 如果games_df为空，直接赋值，否则在行方向上进行合并
    if games_df.empty:
        games_df = subset
    else:
        games_df = pd.concat([games_df, subset], axis=0,ignore_index=True ) 

games_df.columns=['name', 'playtime_forever', 'price', 'genres','developers', 'publishers', 'categories', 'release_date',]    

C:\Users\55386\AppData\Local\Temp\ipykernel_15212\1411383680.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.rename(columns={col:''  for col in new_df.columns}, inplace=True)


In [481]:
# 删除未给出名称的游戏
games_df = games_df[games_df['name']!=('Unknown')]

##### 1. 关键数据整合

In [482]:
grouped = games_df.groupby('name').agg({'playtime_forever':['count','mean','median','max','min','std']})
# 将多级列名展平成单级列名
display(grouped)
display(grouped.columns)
grouped['playtime_forever', 'cv'] = grouped[('playtime_forever', 'std')] / grouped[('playtime_forever', 'mean')]
grouped.columns = ['_'.join(col) for col in grouped.columns]
# 重置索引，使其变为平面格式
grouped.reset_index(inplace=True)
grouped

playtime_forever                                        \
                                count          mean   median     max    min   
name                                                                          
100%OrangeJuice                     6  31540.666667   8249.5  126727      0   
11-11MemoriesRetold                 2      0.000000      0.0       0      0   
12LaboursofHercules                 1      0.000000      0.0       0      0   
1v1.LOL                             2    342.500000    342.5     685      0   
3DCoatModdingTool                   1  14872.000000  14872.0   14872  14872   
...                               ...           ...      ...     ...    ...   
太吾绘卷TheScrollOfTaiwu                1  11926.000000  11926.0   11926  11926   
我来自江湖                               1   1000.000000   1000.0    1000   1000   
暖雪WarmSnow                          1   5762.000000   5762.0    5762   5762   
雀魂麻将(MahjongSoul)                   5   3341.800000   3052.0    6161    906   
鬼谷八荒TaleofImmortal                  5  14674.200000  16642.0   25487   5087   

                                    
                               std  
name                                
100%OrangeJuice       49695.752638  
11-11MemoriesRetold       0.000000  
12LaboursofHercules            NaN  
1v1.LOL                 484.368145  
3DCoatModdingTool              NaN  
...                            ...  
太吾绘卷TheScrollOfTaiwu           NaN  
我来自江湖                          NaN  
暖雪WarmSnow                     NaN  
雀魂麻将(MahjongSoul)      1901.407821  
鬼谷八荒TaleofImmortal     8004.725148  

[2267 rows x 6 columns]

MultiIndex([('playtime_forever',  'count'),
            ('playtime_forever',   'mean'),
            ('playtime_forever', 'median'),
            ('playtime_forever',    'max'),
            ('playtime_forever',    'min'),
            ('playtime_forever',    'std')],
           )

,name,playtime_forever_count,playtime_forever_mean,playtime_forever_median,playtime_forever_max,playtime_forever_min,playtime_forever_std,playtime_forever_cv
0,100%OrangeJuice,6,31540.666667,8249.5,126727,0,49695.752638,1.575609
1,11-11MemoriesRetold,2,0.000000,0.0,0,0,0.000000,NaN
2,12LaboursofHercules,1,0.000000,0.0,0,0,NaN,NaN
3,1v1.LOL,2,342.500000,342.5,685,0,484.368145,1.414214
4,3DCoatModdingTool,1,14872.000000,14872.0,14872,14872,NaN,NaN
...,...,...,...,...,...,...,...,...
2262,太吾绘卷TheScrollOfTaiwu,1,11926.000000,11926.0,11926,11926,NaN,NaN
2263,我来自江湖,1,1000.000000,1000.0,1000,1000,NaN,NaN
2264,暖雪WarmSnow,1,5762.000000,5762.0,5762,5762,NaN,NaN
2265,雀魂麻将(MahjongSoul),5,3341.800000,3052.0,6161,906,1901.407821,0.568977


In [483]:
# 将游戏数据与玩家相关的统计数据融合并保存
grouped = pd.merge(left= grouped, right= games_df,how='left', left_on='name', right_on='name',suffixes=['','_suffixes'],).drop_duplicates(subset='name')
grouped.to_csv('./query_results/most_frequent_play_game_all_timelist.csv')

##### 2. 进行sort,filter等操作，请参考[sql文件](./sort_and_filter_for_most_popular_games_all_time.sql)

##### 3. 根据查询结果进一步分析经典游戏营收

In [484]:
expensive_hot_games = pd.read_csv('./query_results/TOP_100_dollar_to_hours.csv')

In [485]:
genres_dict = {}
count = 1
for i in expensive_hot_games['genres']:
    # 将字符转成应有的数据类型,literal表示字面量，即原本是什么类型，eval表示评估
    for genre in (ast.literal_eval(i)):
        if genre in genres_dict:
            genres_dict[genre]+=1
        else:
            genres_dict[genre]=1

genres_se = pd.Series(genres_dict)
genres_se.sort_values(ascending=False)

Action                  73
Adventure               47
Indie                   24
RPG                     21
Simulation              17
Casual                  13
Strategy                11
Sports                   9
Racing                   8
MassivelyMultiplayer     4
EarlyAccess              3
Design&Illustration      1
Education                1
WebPublishing            1
GameDevelopment          1
dtype: int64

通过结果可以看出动作&冒险&独立类游戏单位时间美元数更多

##### 4. 分析发行商及开发商上榜次数

In [486]:
# 假设已有DataFrame 'grouped' 包含相关信息
developers_df = grouped[['name','playtime_forever_count','developers','publishers','release_date']]

# 使用groupby聚合游戏名称列表
developers_df = developers_df.groupby('developers').agg({
    'name': lambda x: list(x)  # 使用list来获取所有游戏名称
}).sort_values('name', ascending=False)  # 根据游戏数量排序，可根据需要调整

# 重命名列，便于理解
developers_df.rename(columns={'name': 'game_names'}, inplace=True)
developers_df = developers_df.reset_index()

developers_df['count'] = 0 # 初始化数量
# 计算上榜游戏数量
for i in range(len(developers_df)):
    developers_df.iloc[i,2]= len(developers_df.iloc[i,1])

# 显示榜单
developers_df = developers_df.sort_values(by='count',ascending=False)
# 重新排序
developers_df.reset_index(drop=True,inplace=True)

In [487]:
developers_df[ (developers_df['count']>2) & (developers_df['developers'] != 'Unknown') ]

,developers,game_names,count
1,['Valve'],"[ArtifactClassic, Counter-Strike, Counter-Stri...",25
2,['SquareEnix'],"[CHRONOTRIGGER, DRAGONQUESTXIS:EchoesofanElusi...",17
3,"['CAPCOMCo.,Ltd.']","[CapcomArcade2ndStadium, DevilMayCry5, DevilMa...",15
4,['UbisoftMontreal'],"[Assassin'sCreed, Assassin'sCreedBrotherhood, ...",11
5,['SportsInteractive'],"[EastsideHockeyManager, FootballManager2011, F...",11
...,...,...,...
109,['BandaiNamcoStudiosInc.'],"[ACECOMBAT™7:SKIESUNKNOWN, TEKKEN8, TalesofArise]",3
110,"['RelicEntertainment', 'FeralInteractive(Mac/L...","[Warhammer40,000:DawnofWarII, Warhammer40,000:...",3
111,"['VisualConcepts', ""Yuke'sCo.,LTD""]","[WWE2K17, WWE2K18, WWE2K19]",3
112,['SCSSoftware'],"[AmericanTruckSimulator, EuroTruckSimulator2, ...",3


#### 统计近期热门游戏

数据整理

In [488]:
'''读取数据'''
df = pd.read_csv('../../data/processed/all_steam_and_game_data_after_cleaned.csv')

In [489]:
'''提取并整理近期（2周）热门游戏相关信息'''
two_weeks_hot_game_df = df[df.columns[66:]]
two_weeks_hot_game_df.rename(columns={col:'' for col in two_weeks_hot_game_df.columns},inplace=True)

C:\Users\55386\AppData\Local\Temp\ipykernel_15212\2524270102.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  two_weeks_hot_game_df.rename(columns={col:'' for col in two_weeks_hot_game_df.columns},inplace=True)


In [490]:
'''将所有近期热门游戏整理成扁平数据'''

# 获取 DataFrame 的总列数
total_columns = two_weeks_hot_game_df.shape[1]

# 按照每12列分割 DataFrame，并将它们纵向拼接
dfs = [two_weeks_hot_game_df.iloc[:, i:i+12] for i in range(0, total_columns, 12)]
result_df = pd.concat(dfs, ignore_index=True,axis= 0)
result_df.columns = ['appid_game_2weeks', 'name_game_2weeks', 'playtime_2weeks_game',
       'playtime_forever_game_2weeks', 'steam_appid_2weeks_game_',
       'price_overview_2weeks_game_', 'genres_2weeks_game_',
       'developers_2weeks_game_', 'publishers_2weeks_game_',
       'categories_2weeks_game_', 'release_date_2weeks_game_',
       'metacritic_2weeks_game_']

In [491]:
'''清理无效数据'''
result_df = result_df[result_df['appid_game_2weeks'] !=0]
result_df

,appid_game_2weeks,name_game_2weeks,playtime_2weeks_game,playtime_forever_game_2weeks,steam_appid_2weeks_game_,price_overview_2weeks_game_,genres_2weeks_game_,developers_2weeks_game_,publishers_2weeks_game_,categories_2weeks_game_,release_date_2weeks_game_,metacritic_2weeks_game_
3,730,Counter-Strike2,1189,371092,730,0,"['Action', 'FreetoPlay']",['Valve'],['Valve'],"['Multi-player', 'Cross-PlatformMultiplayer', ...","Aug21,2012",No score
10,846770,DYSMANTLE,467,467,846770,19,"['Action', 'Adventure', 'Indie', 'RPG', 'Simul...",['10tonsLtd'],['10tonsLtd'],"['Single-player', 'Multi-player', 'Co-op', 'Sh...","Nov16,2021",No score
11,440,TeamFortress2,1562,273682,440,0,"['Action', 'FreetoPlay']",['Valve'],['Valve'],"['Multi-player', 'Cross-PlatformMultiplayer', ...","Oct10,2007",92
13,730,Counter-Strike2,394,27829,730,0,"['Action', 'FreetoPlay']",['Valve'],['Valve'],"['Multi-player', 'Cross-PlatformMultiplayer', ...","Aug21,2012",No score
24,730,Counter-Strike2,37,157197,730,0,"['Action', 'FreetoPlay']",['Valve'],['Valve'],"['Multi-player', 'Cross-PlatformMultiplayer', ...","Aug21,2012",No score
...,...,...,...,...,...,...,...,...,...,...,...,...
61290,2120250,WarTortoise,877,3454,2120250,9,['Action'],['FoursakenMedia'],['FoursakenMedia'],"['Single-player', 'Fullcontrollersupport', 'St...","27Mar,2024",No score
61306,2357570,Overwatch®2,406,5883,2357570,0,"['Action', 'FreetoPlay']","['BlizzardEntertainment,Inc.']","['BlizzardEntertainment,Inc.']","['Multi-player', 'PvP', 'OnlinePvP', 'Co-op', ...","Aug10,2023",No score
61315,553850,HELLDIVERS™2,237,2063,553850,32,['Action'],['ArrowheadGameStudios'],['PlayStationPCLLC'],"['Multi-player', 'Co-op', 'OnlineCo-op', 'Stea...","8Feb,2024",No score
61321,1938090,CallofDuty®,5,6860,1938090,0,['Action'],"['SledgehammerGames', 'Treyarch', 'InfinityWar...",['Activision'],"['Single-player', 'Multi-player', 'PvP', 'Onli...","27Oct,2022",No score


In [494]:
result_df_statistic = result_df.groupby('name_game_2weeks').agg({
    'playtime_2weeks_game':['count','max','median','mean','min']
}).sort_values(by= ('playtime_2weeks_game', 'count'),ascending= False)
result_df_statistic.columns = ['_'.join(col) for col in result_df_statistic.columns]
result_df_statistic = result_df_statistic.reset_index()

In [505]:
grouped_2weeks = pd.merge(left=result_df_statistic, right=result_df, left_on='name_game_2weeks', right_on='name_game_2weeks', how= 'left' ).drop_duplicates(subset='name_game_2weeks').reset_index(drop=True)
grouped_2weeks.to_csv('./query_results/most_frequent_play_game_2weeks.csv')

In [512]:
grouped_2weeks

,name_game_2weeks,playtime_2weeks_game_count,playtime_2weeks_game_max,playtime_2weeks_game_median,playtime_2weeks_game_mean,playtime_2weeks_game_min,appid_game_2weeks,playtime_2weeks_game,playtime_forever_game_2weeks,steam_appid_2weeks_game_,price_overview_2weeks_game_,genres_2weeks_game_,developers_2weeks_game_,publishers_2weeks_game_,categories_2weeks_game_,release_date_2weeks_game_,metacritic_2weeks_game_
0,Counter-Strike2,1159,18843,418.0,817.949957,1,730,1189,371092,730,0,"['Action', 'FreetoPlay']",['Valve'],['Valve'],"['Multi-player', 'Cross-PlatformMultiplayer', ...","Aug21,2012",No score
1,Dota2,258,8286,1051.5,1354.941860,1,570,2372,567826,570,0,"['Action', 'Strategy', 'FreetoPlay']",['Valve'],['Valve'],"['Multi-player', 'Co-op', 'SteamTradingCards',...","Jul9,2013",90
2,PUBG:BATTLEGROUNDS,221,12370,268.0,674.411765,2,578080,509,26381,578080,0,"['Action', 'Adventure', 'MassivelyMultiplayer'...","['KRAFTON,Inc.']","['KRAFTON,Inc.']","['Multi-player', 'PvP', 'OnlinePvP', 'Stats', ...","Dec21,2017",No score
3,HELLDIVERS™2,202,8465,325.5,615.034653,1,553850,1044,16289,553850,32,['Action'],['ArrowheadGameStudios'],['PlayStationPCLLC'],"['Multi-player', 'Co-op', 'OnlineCo-op', 'Stea...","8Feb,2024",No score
4,ApexLegends,164,6239,492.5,804.969512,4,1172470,2159,53061,1172470,0,"['Action', 'Adventure', 'FreetoPlay']",['Respawn'],['ElectronicArts'],"['Multi-player', 'PvP', 'OnlinePvP', 'Co-op', ...","Nov4,2020",88
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1461,DuckGame,1,19,19.0,19.000000,19,312530,19,6547,312530,12,"['Action', 'Indie']",['LandonPodbielski'],['AdultSwimGames'],"['Single-player', 'Multi-player', 'PvP', 'Onli...","Jun4,2015",82
1462,PeggleNights,1,305,305.0,305.000000,305,3540,305,894,3540,4,['Casual'],"['PopCapGames,Inc.']","['PopCapGames,Inc.', 'ElectronicArts']","['Single-player', 'RemotePlayonPhone', 'Remote...","Oct15,2008",76
1463,Peglin,1,97,97.0,97.000000,97,1296610,97,6275,1296610,19,"['Casual', 'Indie', 'RPG', 'Strategy', 'EarlyA...",['RedNexusGamesInc.'],"['RedNexusGamesInc.', 'IndieArk']","['Single-player', 'SteamAchievements', 'Fullco...","Apr25,2022",No score
1464,DreamEngines:NomadCities,1,962,962.0,962.000000,962,1076750,962,1550,1076750,19,"['Indie', 'RPG', 'Simulation', 'Strategy']",['Suncrash'],"['Suncrash', 'GameraGames']","['Single-player', 'SteamAchievements', 'SteamT...","9May,2024",No score


In [513]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns

# 假设数据集已经加载为df
df = pd.read_csv('customer_data.csv')

# 数据预处理
df.dropna(inplace=True)
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df[['Age', 'Annual Income', 'Spending Score']])

# 确定最优的K值
inertia = []
for k in range(1, 11):
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(df_scaled)
    inertia.append(kmeans.inertia_)

plt.figure(figsize=(10, 6))
plt.plot(range(1, 11), inertia, marker='o')
plt.xlabel('Number of Clusters (K)')
plt.ylabel('Inertia')
plt.title('Elbow Method For Optimal K')
plt.show()

# 假设通过肘部法确定最优K值为3
kmeans = KMeans(n_clusters=3, random_state=42)
df['Cluster'] = kmeans.fit_predict(df_scaled)

# 聚类结果可视化
plt.figure(figsize=(10, 6))
sns.scatterplot(x='Annual Income', y='Spending Score', hue='Cluster', data=df, palette='viridis')
plt.title('Customer Segments')
plt.show()

# 聚类特征分析
cluster_summary = df.groupby('Cluster').mean()
print(cluster_summary)

# 根据分析结果提出营销策略


FileNotFoundError: [Errno 2] No such file or directory: 'customer_data.csv'